In [1]:
import pandas as pd
import os


path = "./dll_analysis/malware"
filename = "fixed_rules.csv"
file_to_scan = os.path.join(path,filename)
df1 = pd.read_csv(file_to_scan)

In [2]:
df1.head()


,Unnamed: 0,Exe name,Family,first_seen,last_seen,total,positives,ssdeep,imphash,yara_detected,yara_matching_rules
0,0,05c0f641bc943f8c1b6c1eddafebc2ee107676b3b2c57a...,locky,2016-11-02 08:42:23,2016-11-02 08:42:23,57.0,32.0,3072:6M0OkZhEYDToI3aS4OHjNobuQih7Pu/X8iJdpxsnL...,cba475024652ed6788f8d77ae1a30155,True,['../yara/rules-master/malware/h3x2b/yara-rule...
1,1,1524b86c0e00ac5b93af3db19265a79ec269ef67573802...,locky,2016-10-31 12:30:02,2016-11-01 19:45:57,57.0,37.0,3072:EH5NBtlf2OkjN9rbNIGD9sIogzNGHPXk8wr8rFnjG...,a23d5ed45b5bb2d45988b4d790bb2fb7,True,['../yara/rules-master/malware/x64dbg/yarasigs...
2,2,d82b557fc694192fd928fbf916a979c0838eec6d42f334...,razy,2016-11-04 16:48:17,2016-11-04 16:48:17,57.0,20.0,3072:Q9FusUCqe04D+Dmo5NdPQ1IYkze2b1OBGcXH3nXPi...,3736395e43e13030dde5ba3bc3da56d5,False,[]
3,3,33fbae9c05b6e46aa3360bd20d2f39216b91b6c167595a...,locky,2016-11-04 14:36:51,2016-11-04 14:36:51,56.0,39.0,3072:mLhD1etmEcttEm7zj/FDVaRj0KmClAnXH3nXPqn:c...,3736395e43e13030dde5ba3bc3da56d5,False,[]
4,4,58114b0f4eb890f498b96eed084c2f341f711dbf9fc941...,locky,2016-11-02 20:17:59,2016-12-06 20:27:17,57.0,44.0,3072:nnpfH5egxO5TIB9FAy1ZsMKNDO5kwMAFbjSlWmVrT...,9cd36be5889827aef4f69402aa78a2ba,False,[]


In [3]:
import pefile
import ssdeep

m1 = df1.loc[:,["Exe name", "Family", "first_seen", "ssdeep"]]
m1["first_seen:month"] = pd.to_datetime(m1["first_seen"]).dt.to_period("M")

In [4]:
del m1["first_seen"]

In [5]:
m1.head()

,Exe name,Family,ssdeep,first_seen:month
0,05c0f641bc943f8c1b6c1eddafebc2ee107676b3b2c57a...,locky,3072:6M0OkZhEYDToI3aS4OHjNobuQih7Pu/X8iJdpxsnL...,2016-11
1,1524b86c0e00ac5b93af3db19265a79ec269ef67573802...,locky,3072:EH5NBtlf2OkjN9rbNIGD9sIogzNGHPXk8wr8rFnjG...,2016-10
2,d82b557fc694192fd928fbf916a979c0838eec6d42f334...,razy,3072:Q9FusUCqe04D+Dmo5NdPQ1IYkze2b1OBGcXH3nXPi...,2016-11
3,33fbae9c05b6e46aa3360bd20d2f39216b91b6c167595a...,locky,3072:mLhD1etmEcttEm7zj/FDVaRj0KmClAnXH3nXPqn:c...,2016-11
4,58114b0f4eb890f498b96eed084c2f341f711dbf9fc941...,locky,3072:nnpfH5egxO5TIB9FAy1ZsMKNDO5kwMAFbjSlWmVrT...,2016-11


In [6]:
""" Removing samples with empty ssdeep values """
m1 = m1.dropna()

In [7]:
import ssdeep

In [8]:
UniqueFamilies = m1.Family.unique()

In [9]:
df_dict={el: pd.DataFrame() for el in UniqueFamilies}

In [10]:
for family in UniqueFamilies:
    mf = m1[:][m1.Family == family]
    """ Performing cartesian product of the dataset with itself
    ( it will be useful to use the apply function over the same row of the dataframe).
    Each row will now contain two samples of the same family 
    (too complex and, probably, useless to perform it over all the samples, independently on the family)"""
    cp = mf.merge(mf, how='outer', on = "Family")
    cp = cp.dropna()
    cp["Similarity"] = cp.apply(lambda x: ssdeep.compare(str(x["ssdeep_x"]),str(x["ssdeep_y"])), axis=1)
    df_dict[family] = cp


Exe name_y,0045a8f95a7c6d204b42f003bc4779293ba32abccfbc34bb4c81badbe768d975,00ed72ffa97f75e1f4d1769e25f94614dbf3fe2a468361b3511a7c0b10ba9038,0116ab9e124e84ba324f2560b882491970d0861216b542cb0ba691780ffea4a2,012a23fc7bab2e5f4c3656d0574e70b47add09b2cdbe40dc33e45393bb4180c5,02cdac1d4c67c3b3b02bb969ac19646d9857d5d511109f12628ba7aba69ce6ca,030dfdd376df50875e6e1a4ab5a103dc24cb6d014e975f928b84bfec669d2933,043b95cd48d45ae21e40f670846a157ddc16757422c0a22b61b6d0775f68f009,047e6c349a6aa6c729685ba09c6127f022e35f5e51de8f2a396df3bf4d74c57e,050fb87de79b6e4d9c83d71b586b211855be6460c4daddf2ecb41580c32f6b36,053f93b3645f4e900af75241e6695003f86cd10abb5a427537679567d5798a0e,...,fa0c084aef41969f7b1f427a1b65e1b1464fa82d297e712108e1fdf312c04481,fa0f85ae06ed399c32e61906a94a3a233d68a8aed38281fb9697d274d16e20f3,facfeca271ff2aacf78e97442d1ec77151e704f4801bb06eb7c5d03366db636c,faedd03a6d25ae12eaf93442dde0af307671ccd871a4d2e9a3c5441eb7ba85e8,fbbd773bbf0e4da2158740a157d941b95a0c40afbf333a943266843d51b58e5e,fbfcc0d37e41b68a4c76fe013bdcad52f546435cd0b58de8fa0ca58b7b3bb7f3,fc59d02c0aa136dbbcfab19ba944d28f70f77ed2def6c5b3dfb75f6d5ad6999f,fd08ed15fbb2536908a0018b68c30f5ae0ba2baa2835d785db922a7178a0639d,fe621e0157e6eccf323a6ab9616277f7bcf437704a1bf5aa889ffae96d2460a9,feeeadabca104f5fad1acafe6b164a485c732c69c5852874784a97d5a761f0b3
Exe name_x,,,,,,,,,,,,,,,,,,,,,
0018bc9f3e36c70fd4abf0171a5be8edcedc79b86ab19884438d6560870af59f,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01722a55b090ab9e13cfae2b6ed0d9ed53199f430eabb7572441b94ade623834,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
017c1050e8ed1dbff0e05d617701ac25264f890b97181041b56d19b14b01229f,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
029267049e4495414a36adb7383a0602c49285d98e4e261d290171994ca8d3bb,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
02be6fcee1107b3b0cc6c325203a946362b410bfb9933fd4998897800bf5142f,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
import matplotlib.pyplot as plt
for family in UniqueFamilies:
    if(df_dict[family].shape[0]<2):
        continue
    df_dict[family] = df_dict[family].pivot(index= "Exe name_x", columns="Exe name_y", values="Similarity")
    ax = plt.pcolor(df_dict[family])
    ax.get_figure().savefig(path+"/heatmap/ssdeep/"+family+"_heatmap", dpi = 700)
    plt.clf()

In [12]:
for family in UniqueFamilies:
    df_dict[family].to_csv(path+"/heatmap/ssdeep/"+family+"_heatmap.csv")